### 1.Importing Libraries

In [40]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
flights = pd.read_csv('flight_price.csv')
flights


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [3]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- Data set contains 10683 rows and 11 features
- Columns `Route` and `Total_Stops` have missing values
- The Data types of some features are not appropriate

In [4]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [5]:
value = flights.Date_of_Journey.iloc[6]
value

'12/03/2019'

### Observations

- The type of `Date_of_Journey`, `Dep_Time` and `Arrival_Time` should be datetime.
- The types of `Duration` and `Total_Stops` are mixed. They should be numaric type.
- There ar 220 duplicates. They should be removed.



In [6]:
flights.duplicated().sum()

np.int64(220)

In [7]:
(
    flights
    .loc[flights.duplicated(keep = False)]
    .sort_values(["Airline","Date_of_Journey","Source","Destination","Route","Dep_Time","Arrival_Time","Duration","Total_Stops","Additional_Info"])
).sample()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9110,Jet Airways,24/06/2019,Delhi,Cochin,DEL → AMD → BOM → COK,19:10,19:00 25 Jun,23h 50m,2 stops,No info,12819


In [8]:
def clean_data(df):
    return (
        df.assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include = 'O').columns
        })
        .rename(columns=str.lower)
        .assign(
            airline = lambda df_ :(
                df_
                .airline
                .str.replace(" Premium economy","")
                .str.replace(" Business","")
                .str.title()
                
            )
        )
    )

- here we have written the lamda function to get the current state of the data frame.
- That means, without lamda there will be error to column name that "airline" does not exist in df.
- As df has column "Airline" not "airline".
- each pandas function is makenew copy, never override the original copy of the data frame.
- "airline = lambda df_: df_.airline.str.lower()"
- " select_dtypes(include = 'O')" this will give us all columns which are of object type, hence pass big o in double quoates.
- In above key is column name `col` and value is operation `df[col].str.strip()`
- To know more read `Dictionary Comprehension` and `Dictionary Unpacking`

In [9]:
clean_data(flights).airline.unique()

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

In [10]:
flights.select_dtypes(include = 'O').columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info'],
      dtype='object')

In [11]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [12]:
(
    flights
    .Airline
    .str.replace(" Premium economy","")
    .str.replace(" Business","")
    .str.title()
).unique()

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

#### Some of the entries have inconsistent / inaccurate values

In [13]:
pd.to_datetime(flights.Arrival_Time, dayfirst= True)

C:\Users\ajink\AppData\Local\Temp\ipykernel_18792\3748028143.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Arrival_Time, dayfirst= True)


0       2025-03-22 01:10:00
1       2025-01-05 13:15:00
2       2025-06-10 04:25:00
3       2025-01-05 23:30:00
4       2025-01-05 21:35:00
                ...        
10678   2025-01-05 22:25:00
10679   2025-01-05 23:20:00
10680   2025-01-05 11:20:00
10681   2025-01-05 14:10:00
10682   2025-01-05 19:15:00
Name: Arrival_Time, Length: 10683, dtype: datetime64[ns]

In [14]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [15]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

- #### Observe that Route column has nothing but only redundant information. i.e. starting stop, ending stop and number of stopes in between.

In [16]:
(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    # finding out such rows which has any character apart form 0-9 and :.
    # .str.split(" ",n = 1) # split on first space

)

Series([], Name: Dep_Time, dtype: object)

In [17]:
pd.to_datetime(flights.Dep_Time).dt.time

C:\Users\ajink\AppData\Local\Temp\ipykernel_18792\1056054517.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival Time

In [18]:
(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ",n = 1)
    .str.get(1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

### Duration

In [19]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [20]:
(
    flights
    .Duration
    .loc[ lambda ser: ~ser.str.contains("m")] # rows which does not contain m
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [21]:
(
    flights
    .Duration
    .loc[ lambda ser: ~ser.str.contains("h")] # rows which does not contain h
    # .unique()
)

6474    5m
Name: Duration, dtype: object

In [22]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


The observation index 6474 has duration of 5 min this is clearly wrong. Will delete this observation.

In [23]:
(
    flights
    .Duration
    .drop(index = 6474)
    .str.split(" ",expand=True) # This will now give us dataframe with 2 columns
    .set_axis(["hours","minutes"],axis = 1)
    .assign(
        hours = lambda df_ : (
            df_
            .hours
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minutes = lambda df_ : (
            df_
            .minutes
            .str.replace("m","")
            .fillna(0)
            .astype(int)
        )

    )
    .sum(axis = 1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [24]:
def convert_to_minutes(ser):
    return(
        ser
        .str.split(" ",expand=True) # This will now give us dataframe with 2 columns
        .set_axis(["hours","minutes"],axis = 1)
        .assign(
        hours = lambda df_ : (
            df_
            .hours
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minutes = lambda df_ : (
            df_
            .minutes
            .str.replace("m","")
            .fillna(0)
            .astype(int)
        )

    )
    .sum(axis = 1)
        
    )

### Total_Stops

In [25]:
(
    flights
    .Total_Stops
    .replace("non-stop","0")
    .str.replace(" stops?","",regex = True)
    # .str.replace(" stops","")
    # .str.replace(" stop","")
    .pipe(lambda ser : pd.to_numeric(ser)) # converted into float
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

In [26]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

### additional_info

In [27]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [28]:
def clean_data(df):
    return (
        df
        .drop(index = [6474])
        .drop_duplicates()
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include = 'O').columns
        })
        .rename(columns=str.lower)
        .assign(
            airline = lambda df_ :(
                df_
                .airline
                .str.replace(" Premium economy","")
                .str.replace(" Business","")
                .str.title()     
            ),
            date_of_journey = lambda df_ : pd.to_datetime(df.Date_of_Journey, dayfirst= True),
            dep_time = lambda df_ : pd.to_datetime(df_.dep_time).dt.time,
            arrival_time = lambda df_ : pd.to_datetime(df_.arrival_time).dt.time,
            duration = lambda df_: df_.duration.pipe(convert_to_minutes), # same as `convert_to_minutes(df_.duration)`
            total_stops = lambda df_ : (
                df_
                .total_stops
                .replace("non-stop","0")
                .str.replace(" stops?","",regex = True)
                .pipe(lambda ser : pd.to_numeric(ser))
            ),
            additional_info = lambda df_ : df_.additional_info.replace('No info','No Info')
        ).drop(columns = "route")
    )

In [29]:
flights_cleaned = clean_data(flights)
flights_cleaned

C:\Users\ajink\AppData\Local\Temp\ipykernel_18792\51101612.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time = lambda df_ : pd.to_datetime(df_.dep_time).dt.time,
C:\Users\ajink\AppData\Local\Temp\ipykernel_18792\51101612.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time = lambda df_ : pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


### Splitting the data

In [34]:
X = flights_cleaned.drop(columns="price")
y = flights_cleaned.price.copy()

In [35]:
X_, X_test, y_, y_test = train_test_split(X,y,test_size=0.2, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_,y_,test_size = 0.2,random_state=42)


In [38]:
Data_DIR = r"C:\Users\ajink\Desktop\CampusX____FLASK\part8"


In [41]:
def export_data(X,y,name):
    file_name = f"{name}.csv"
    file_path = os.path.join(Data_DIR, file_name)

    X.join(y).to_csv(file_path, index = False)

    return pd.read_csv(file_path).head()


In [42]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-21,Banglore,New Delhi,08:55:00,19:10:00,615,1.0,In-flight meal not included,7832
1,Jet Airways,2019-03-27,Delhi,Cochin,17:30:00,04:25:00,655,1.0,In-flight meal not included,6540
2,Goair,2019-03-09,Banglore,New Delhi,11:40:00,14:35:00,175,0.0,No Info,7305
3,Air India,2019-06-12,Kolkata,Banglore,09:25:00,18:30:00,545,1.0,No Info,8366
4,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,07:40:00,525,1.0,In-flight meal not included,11087


In [43]:
export_data(X_val, y_val,"val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-24,Delhi,Cochin,20:25:00,01:30:00,305,1.0,No Info,5054
1,Multiple Carriers,2019-06-12,Delhi,Cochin,09:45:00,22:30:00,765,1.0,No Info,9646
2,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,15:15:00,980,1.0,In-flight meal not included,11087
3,Multiple Carriers,2019-06-06,Delhi,Cochin,13:00:00,21:00:00,480,1.0,No Info,13587
4,Jet Airways,2019-05-18,Delhi,Cochin,23:05:00,04:25:00,1760,2.0,No Info,16704


In [44]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Banglore,New Delhi,08:00:00,08:15:00,1455,1.0,No Info,17996
1,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0.0,No Info,3873
2,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0.0,No Info,4462
3,Indigo,2019-06-27,Chennai,Kolkata,19:35:00,21:55:00,140,0.0,No Info,3597
4,Indigo,2019-05-06,Kolkata,Banglore,15:15:00,17:45:00,150,0.0,No Info,4804
